<a href="https://colab.research.google.com/github/aleksandraqubert/-2/blob/main/Laboratorna3_4kyrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
number_cores = 2
memory_gb = 4
is_full_dataset = True
from pyspark import SparkConf, SparkContext
import os

number_cores = 2
memory_gb = 4
conf = (
    SparkConf()
        .setAppName("Spark Rdd Task")
        .setMaster(f'local[{number_cores}]')
        .set('spark.driver.memory', f'{memory_gb}g')
)

sc = SparkContext.getOrCreate(conf=conf)

if is_full_dataset:
    if not os.path.exists('Reviews.csv'):
        sc.stop()
        raise Exception("""
            Download the 'Reviews.csv' file from https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
            and put it in 'input' folder
        """)
    else:
        inputRdd = sc.textFile("Reviews.csv")
else:
    inputRdd = sc.textFile("sample.csv")
# Видалення рядку заголовку
filteredInput = inputRdd.filter(lambda line: line.startswith("Id,") == False)
# Ваш код починається тут
# працювати треба з RDD filteredInput

    # Розділяємо кожен рядок на поля
split_fields = filteredInput.map(lambda row: row.split(','))

# Створюємо пари (`UserId` та `ProductId`)
user_product_pairs = split_fields.map(lambda own: (own[2], own[1]))

# Групуємо за Користувачем та об'єднуємо всі Товари до списку унікальних продуктів
unique_products_per_user = user_product_pairs.groupByKey().mapValues(lambda product: list(set(product)))
unique_products_per_user.take(10)
from itertools import combinations
# Функція для створення пар продуктів та підрахунку їх
def get_product_pairs(products):
    product_pairs = list(combinations(products, 2))  # Усі можливі комбінації товарів
    return [(pair, 1) for pair in product_pairs]

# Створюємо всі пари продуктів для кожного користувача
user_product_pairs = unique_products_per_user.flatMap(lambda user_products: get_product_pairs(user_products[1]))

# Вивовдимо результат
user_product_pairs.take(10)

from itertools import combinations
from collections import Counter

# Функція для створення пар товарів та підрахунку їх кількості
def get_product_pairs(products):
    product_pairs = list(combinations(products, 2))  # Всі можливі комбінації товарів
    return Counter(product_pairs)  # Підрахунок кількості кожної пари товарів

# Створюємо всі пари товарів для кожного користувача та підраховуємо їх кількість
product_pairs_count = unique_products_per_user.flatMapValues(get_product_pairs)\
                                               .map(lambda x: (x[1], 1))\
                                               .reduceByKey(lambda x, y: x + y)

# Сортуємо пари продуктів за кількістю від найбільшої до найменшої
sorted_product_pairs_count = product_pairs_count.sortBy(lambda x: x[1], ascending=False)

# Беремо лише перші 10 пар продуктів та їх кількість
top_10_product_pairs_count = sorted_product_pairs_count.take(10)

# Виводимо результат
top_10_product_pairs_count

# Шлях до файлу, в який ми збережемо результат
output_path = "output_pairs_product.txt"

# Зберігаємо результат у текстовий файл
sorted_product_pairs_count.map(lambda x: f"{x[0]}: {x[1]}")\
                          .saveAsTextFile(output_path)
sc.stop()